In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
import shap

In [2]:
data = pd.read_csv('../data/LLCP_agg_cleaned.csv')

In [3]:
std_scaler = StandardScaler()
X = data.drop('CVDCRHD4', axis=1)
y = data['CVDCRHD4']
print(X.columns, X.shape)

X = std_scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Index(['GenHealth', 'ASTHMA3', 'EXERANY2', 'CHCSCNCR', '_BMI5CAT', '_SMOKER3',
       'DRNKANY5', '_AGEG5YR', 'Race_1.0', 'Race_2.0', 'Race_3.0', 'Race_4.0',
       'Race_5.0', 'Race_6.0', 'Kidney', 'Bronchitis', 'Depression',
       'Diabetes', 'Sex', 'Heart Attack', 'Cholesterol', 'Stroke',
       'Told Cholesterol High', 'Arthritis'],
      dtype='object') (2127204, 24)
(1701763, 24) (425441, 24) (1701763,) (425441,)


Apply a vanilla kNN model to the data and see how it performs. This is a baseline model to compare against.